##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: JSON Mode Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

The Gemini API can be used to generate a JSON output if you set the schema that you would like to use.

Two methods are available. You can either set the desired output in the prompt or supply a schema to the model separately.

### Install dependencies

In [1]:
%pip install -qU 'google-genai'

Note: you may need to restart the kernel to use updated packages.


In [2]:
from google import genai

import json
import typing_extensions as typing

### Configure your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Set your constrained output in the prompt

For this first example just describe the schema you want back in the prompt:


In [4]:
prompt = """
  List a few popular cookie recipes using this JSON schema:

  Recipe = {'recipe_name': str}
  Return: list[Recipe]
"""

 Activate JSON mode by specifying `respose_mime_type` in the `config` parameter:

In [5]:
MODEL_ID="gemini-2.5-flash-preview-04-17" # @param ["gemini-2.5-flash-preview-04-17", "gemini-2.5-pro-preview-03-25", "gemini-2.0-flash", "gemini-2.0-flash-lite"] {"allow-input":true, isTemplate: true}

raw_response = client.models.generate_content(
    model=MODEL_ID,
    contents= prompt,
    config={
        'response_mime_type': 'application/json'
    },
)

Parse the string to JSON:

In [8]:
response = json.loads(raw_response.text)
print(response)

[{'recipe_name': 'Chocolate Chip Cookies'}, {'recipe_name': 'Oatmeal Raisin Cookies'}, {'recipe_name': 'Peanut Butter Cookies'}, {'recipe_name': 'Sugar Cookies'}, {'recipe_name': 'Shortbread Cookies'}]


For readability serialize and print it:

In [9]:
print(json.dumps(response, indent=4))

[
    {
        "recipe_name": "Chocolate Chip Cookies"
    },
    {
        "recipe_name": "Oatmeal Raisin Cookies"
    },
    {
        "recipe_name": "Peanut Butter Cookies"
    },
    {
        "recipe_name": "Sugar Cookies"
    },
    {
        "recipe_name": "Shortbread Cookies"
    }
]


## Supply the schema to the model directly

The newest models (1.5 and beyond) allow you to pass a schema object (or a python type equivalent) directly and the output will strictly follow that schema.

Following the same example as the previous section, here's that recipe type:

In [13]:
class Recipe(typing.TypedDict):
    recipe_name: str
    recipe_description: str
    recipe_ingredients: list[str]
    recipe_directions: list[str]

For this example you want a list of `Recipe` objects, so pass `list[Recipe]` to the `response_schema` field of the `config`.

In [14]:
result = client.models.generate_content(
    model= MODEL_ID,
    contents="List a few imaginative cookie recipes along with a one-sentence description as if you were a gourmet restaurant and their main ingredients",
    config={
        'response_mime_type': 'application/json',
        'response_schema': list[Recipe],
    },
)

In [15]:
print(json.dumps(json.loads(result.text), indent=4))

[
    {
        "recipe_name": "Rosemary & Olive Oil Shortbread with Lemon Zest",
        "recipe_description": "A delicate shortbread, kissed with fragrant rosemary and bright lemon, offering a sophisticated play of savory and citrus notes.",
        "recipe_ingredients": [
            "Unsalted Butter",
            "All-Purpose Flour",
            "Granulated Sugar",
            "Fresh Rosemary",
            "Extra Virgin Olive Oil",
            "Lemon Zest",
            "Sea Salt"
        ],
        "recipe_directions": [
            "Cream butter and sugar until light and fluffy.",
            "Add olive oil, lemon zest, finely chopped rosemary, and salt.",
            "Gradually mix in flour until dough forms.",
            "Chill dough, then roll and cut into desired shapes.",
            "Bake at 325\u00b0F (160\u00b0C) until edges are lightly golden, about 15-20 minutes.",
            "Let cool completely on a wire rack before serving.",
            "Store in an airtight contai

It is the recommended method if you're using a compatible model.

## Next Steps
### Useful API references:

Check the [structured ouput](https://ai.google.dev/gemini-api/docs/structured-output) documentation or the [`GenerationConfig`](https://ai.google.dev/api/generate-content#generationconfig) API reference for more details

### Related examples

* The constrained output is used in the [Text summarization](../examples/json_capabilities/Text_Summarization.ipynb) example to provide the model a format to summarize a story (genre, characters, etc...)
* The [Object detection](../examples/Object_detection.ipynb) examples are using the JSON constrained output to uniiformize the output of the detection.

### Continue your discovery of the Gemini API

JSON is not the only way to constrain the output of the model, you can also use an [Enum](../quickstarts/Enum.ipynb). [Function calling](../quickstarts/Function_calling.ipynb) and [Code execution](../quickstarts/Code_Execution.ipynb) are other ways to enhance your model by either using your own functions or by letting the model write and run them.